# Comparison of Florianopolis Geometry
Multiple climate zones, comparison of CS 5-zone, umi and surrogate

In [1]:
# import plotly
import plotly.express as px
import pandas as pd
import numpy as np
import os
from pathlib import Path
from geomeppy.patches import EpBunch
from utils.schedules import get_schedules
from utils.constants import *
import json
from archetypal import IDF, BuildingTemplate
from archetypal.umi_template import UmiTemplateLibrary
from umi.ubem import init_template

run_name = "CSboston"
ref_buildings_path = "D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Building Energy References/ReferenceBuildings"
results_dir = "D:/DATA/FlorianopolisBase/BostonBaseline"
shallow_results_dir = "D:/DATA/FlorianopolisBase/BostonShallow"
# guids = pd.read_csv(Path(results_dir) / "guids.csv", header=None)
# Get grasshopper json data for config
gh_data_dir = Path(results_dir) / "SingleBuildings" / f"epjsonsettings_{run_name}_ParametricZoneSettings.json"

c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.3, the latest is 0.3.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


[Taichi] version 1.6.0, llvm 15.0.1, commit f1c6fbbd, win, python 3.8.16
[Taichi] Starting on arch=cuda


In [2]:
month_map = {
    "january": 744,
    "february": 672,
    "march": 744,
    "april": 720,
    "may": 744,
    "june": 720,
    "july": 744,
    "august": 744,
    "september": 720,
    "october": 744,
    "november": 720,
    "december": 744,
}
sum(list(month_map.values()))

8760

## Umi results

In [32]:
# Import umi results for 4A (boston)
dfs = []
for load in ["heating", "cooling"]:
    df = pd.read_csv(Path(results_dir, f"UMI_florianopolis_test_bos_{load}.csv"))
    df["load"] = load
    dfs.append(df)
umi_df = pd.concat(dfs)

In [33]:
umi_df["annual"] = umi_df[[str(x) for x in range(1, 8761)]].sum(axis=1)

In [102]:
umi_df.head()

,Building Id,Building Name,Metric,Units,Resolution,1,2,3,4,5,...,8753,8754,8755,8756,8757,8758,8759,8760,load,annual
0,03b1d5a3-ebbf-4560-a399-b81bc391f9f7,03b1d5a3-ebbf-4560-a399-b81bc391f9f7,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,8803.872406
1,06e706f2-dcac-4c99-a4da-0afbdba2aa9a,06e706f2-dcac-4c99-a4da-0afbdba2aa9a,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,32649.884940
2,0b48b5a6-e91b-4fc9-aeaf-5805a420a8e1,0b48b5a6-e91b-4fc9-aeaf-5805a420a8e1,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,2338.561208
3,0dabd728-d95f-40d5-8836-eba55eba02c2,0dabd728-d95f-40d5-8836-eba55eba02c2,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,2219.940068
4,0e188b41-61da-4d8d-aaf3-be4502ac3cb7,0e188b41-61da-4d8d-aaf3-be4502ac3cb7,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,4282.442673


In [34]:
def get_monthly(df):
    h = df.copy()
    h.set_index(["Building Id", "load"], inplace=True)
    nums = h[[str(x) for x in range(1, 8761)]]
    nums.columns = nums.columns.astype("int")
    c = 0
    print(nums.shape)
    for month, hours in month_map.items():
        h[month] = nums.iloc[:, c : c + hours].sum(axis=1)
        c += hours
    return h[list(list(list(month_map.keys())))]

In [35]:
umi_all_monthly = get_monthly(umi_df).groupby("load").sum()
px.bar(umi_all_monthly.T, x=umi_all_monthly.T.index, y=["cooling", "heating"], title="Long-Form Input")

(186, 8760)


## CS Results (with shading)

In [35]:
# CS with shading
with open(Path(results_dir) / f"results-out_{run_name}_all.json", "r") as f:
# with open(Path("D:/DATA/FlorianopolisBase/BostonBaseline/results-out-CS_boston.json"), "r") as f:
    CS_results = json.load(f)
CS_results = pd.DataFrame.from_dict(CS_results)
CS_results

,heating,cooling
0,37.782502,0.651997
1,30.815120,0.864720
2,23.912920,1.388140
3,8.860720,3.217566
4,1.372989,6.767600
5,0.100272,16.316632
6,0.000004,24.082650
7,0.000000,22.895837
8,0.074960,11.990431
9,3.945830,4.262115


In [36]:
# CS individual buildings
files = [x for x in os.listdir(Path(results_dir) / "SingleBuildings") if "results-out" in x]
assert len(files) == 93

CS_building_results = {}
for fname in files:
    building_id = fname.split("_")[-1].split(".json")[0]
    with open(Path(results_dir) / "SingleBuildings" / fname, "r") as f:
        r = json.load(f)
        res_dict = {}
        for l in ["heating", "cooling"]:
            res_dict.update({(l, k): r for k, r in zip(month_map.keys(), r[l])})
        CS_building_results[building_id] = res_dict
CS_building_results = pd.DataFrame.from_dict(CS_building_results).T
# CS_building_results['guid']= guids
# CS_building_results.set_index('guid', inplace=True)
CS_building_results.head()

heating                        \
                                         january   february      march   
017ecc40-ddbf-481c-9f31-b8e0bc37d4bd  111.650910  90.942701  71.758115   
02e58b5c-19bf-4681-ad73-caaf40755166   78.226751  63.373139  50.192400   
05e64858-d58d-4e02-aace-5792e97c8cea   75.364362  61.143618  48.031179   
094b90f3-f000-4e23-8deb-37ca43f1c3d7   99.310705  81.974780  66.540557   
0a9a2f97-6ff6-46e2-938d-9f17e00f083f   76.586732  62.323101  49.044948   

                                                                          \
                                          april       may      june july   
017ecc40-ddbf-481c-9f31-b8e0bc37d4bd  28.140261  5.287326  0.556407  0.0   
02e58b5c-19bf-4681-ad73-caaf40755166  18.511447  2.496971  0.154491  0.0   
05e64858-d58d-4e02-aace-5792e97c8cea  17.433406  2.213414  0.123035  0.0   
094b90f3-f000-4e23-8deb-37ca43f1c3d7  26.964513  4.611036  0.308343  0.0   
0a9a2f97-6ff6-46e2-938d-9f17e00f083f  17.951447  2.294593  0.128449  0.0   

                                                                  ...  \
                                     august september    october  ...   
017ecc40-ddbf-481c-9f31-b8e0bc37d4bd    0.0  0.387711  15.157001  ...   
02e58b5c-19bf-4681-ad73-caaf40755166    0.0  0.036622   8.636031  ...   
05e64858-d58d-4e02-aace-5792e97c8cea    0.0  0.035377   8.062470  ...   
094b90f3-f000-4e23-8deb-37ca43f1c3d7    0.0  0.253258  13.484575  ...   
0a9a2f97-6ff6-46e2-938d-9f17e00f083f    0.0  0.040403   8.355381  ...   

                                       cooling                       \
                                         march     april        may   
017ecc40-ddbf-481c-9f31-b8e0bc37d4bd  1.034355  4.165585  11.300644   
02e58b5c-19bf-4681-ad73-caaf40755166  0.533748  2.301947   5.988246   
05e64858-d58d-4e02-aace-5792e97c8cea  0.543592  2.113125   6.371521   
094b90f3-f000-4e23-8deb-37ca43f1c3d7  0.200386  1.265397   3.818800   
0a9a2f97-6ff6-46e2-938d-9f17e00f083f  0.406040  1.952692   6.410223   

                                                                       \
                                           june       july     august   
017ecc40-ddbf-481c-9f31-b8e0bc37d4bd  30.753820  45.365826  42.817734   
02e58b5c-19bf-4681-ad73-caaf40755166  22.965712  36.666122  34.475236   
05e64858-d58d-4e02-aace-5792e97c8cea  23.019585  36.436533  34.268420   
094b90f3-f000-4e23-8deb-37ca43f1c3d7  19.758603  33.922338  31.578789   
0a9a2f97-6ff6-46e2-938d-9f17e00f083f  23.243016  36.880518  34.728428   

                                                                               
                                      september   october  november  december  
017ecc40-ddbf-481c-9f31-b8e0bc37d4bd  21.273434  5.586400  2.995909  0.586115  
02e58b5c-19bf-4681-ad73-caaf40755166  14.995683  3.799237  2.027236  0.298786  
05e64858-d58d-4e02-aace-5792e97c8cea  14.919657  3.360987  1.753602  0.313343  
094b90f3-f000-4e23-8deb-37ca43f1c3d7  12.237198  2.728774  1.337415  0.151048  
0a9a2f97-6ff6-46e2-938d-9f17e00f083f  15.104905  3.237357  1.525407  0.236591  

[5 rows x 24 columns]

In [37]:
CS_building_results.loc[np.isnan(CS_building_results['heating']['january'])]

heating                                \
                                     january february march april may june   
73ddd747-14ca-4769-b39a-d8ca915b9846     NaN      NaN   NaN   NaN NaN  NaN   

                                                                    ...  \
                                     july august september october  ...   
73ddd747-14ca-4769-b39a-d8ca915b9846  NaN    NaN       NaN     NaN  ...   

                                     cooling                             \
                                       march april may june july august   
73ddd747-14ca-4769-b39a-d8ca915b9846     NaN   NaN NaN  NaN  NaN    NaN   

                                                                          
                                     september october november december  
73ddd747-14ca-4769-b39a-d8ca915b9846       NaN     NaN      NaN      NaN  

[1 rows x 24 columns]

## Model results

In [38]:
# Load template settings from rhino
with open(gh_data_dir, 'r') as f:
    gh_data = json.load(f)
COP_COOLING = 3.32337
COP_HEATING = 0.75

In [18]:
# Load in GH data for custom template
if gh_data["Conditioning"]["HeatRecoveryType"] == "None":
    hrv = 0
else:
    hrv = HRV[gh_data["Conditioning"]["HeatRecoveryType"]].value 
if "occ" in gh_data["Conditioning"]["MechVentSchedule"].lower():
    vent_mode = 2
else:
    vent_mode = 0 #TODO
rval = 4.88
config = {
    "FacadeMass": 0, #TODO
    "RoofMass": 0,
    "FacadeRValue": rval,
    "RoofRValue": rval,
    "SlabRValue": rval,
    "WindowUValue": float(gh_data["SimpleWindow"]["uval"]),
    "WindowShgc": float(gh_data["SimpleWindow"]["shgc"]),
    "PeopleDensity": float(gh_data["Loads"]["PeopleDensity"]),
    "EquipmentPowerDensity": float(gh_data["Loads"]["EquipmentPowerDensity"]),
    "LightingPowerDensity": float(gh_data["Loads"]["LightingPowerDensity"]),
    "HeatingSetpoint": float(gh_data["Conditioning"]["HeatingSetpoint"]),
    "CoolingSetpoint": float(gh_data["Conditioning"]["CoolingSetpoint"]),
    "cop_heating": 1,
    "cop_cooling": 1,
    "VentilationMode": vent_mode,
    "VentilationPerArea": float(gh_data["Conditioning"]["MinFreshAirArea"])/1000,
    "VentilationPerPerson": float(gh_data["Conditioning"]["MinFreshAirPerson"])/1000,
    "EconomizerSettings": Econ[gh_data["Conditioning"]["EconomizerType"]].value,
    "RecoverySettings": hrv,
    "Infiltration": float(gh_data["Infiltration"]["InfiltrationFlowPerExteriorSurfaceArea"]),
    # "wwr": gh_data["ShoeboxConfig"]["wwr"],
    # "roof_2_footprint": gh_data["ShoeboxConfig"]["roof_2_footprint"],
    # "ground_2_footprint": gh_data["ShoeboxConfig"]["ground_2_footprint"],
    # "orientation": gh_data["ShoeboxConfig"]["orientation"],
    # "width": gh_data["ShoeboxConfig"]["width"],
    # "height": gh_data["ShoeboxConfig"]["height"],
    # "perim_depth": gh_data["ShoeboxConfig"]["perim_depth"],
    # "core_depth": gh_data["ShoeboxConfig"]["core_depth"],
    # "building_idx": 0,
    # "edge_length": 1,
    # "edge_weight": 1,
    # "weight": 1,
    "template_idx": 0,
    "template_name": "ParametricZoneSettings",
    # "floor_name": "none",
}
for i in range(12):
    config[f"shading_{i}"] = 0

In [26]:
import geopandas as gpd
from ladybug.epw import EPW

gdf = gpd.read_file(
    Path("D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Florianopolis")
    / "Florianopolis_Baseline.zip"
)

# dict to store key fields for known gis files
id_cols = {
    "florianpolis": {
        "height_col": "HEIGHT",
        "id_col": "OBJECTID",
        "template_name_col": "template_name",
        "wwr_col": "wwr",
    }
}

"""
Select EPW
"""
# epw_fp = (
#     Path(
#         "D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Building Energy References/ReferenceBuildings"
#     )
#     / "USA_FL-MIAMI.722020_TMY2-12839.epw"
# )
epw_fp = ("C:/ProgramData/Solemma/Common/WeatherData/EPW/USA_MA_Boston-Logan.Intl.AP.725090_TMYx.2004-2018.epw")
epw = EPW(epw_fp)


"""
Make Schedules
"""
schedules = pd.read_hdf("data/schedules.h5")
idx = gh_data["ShoeboxConfig"]["schedule_index"]
print("Schedule index:", idx)
selected_schedule = schedules.loc[schedules.index[idx*3][0]].to_numpy()
"""
Template lib as tuple
"""
template_df = pd.Series(config)
template_df = pd.to_numeric(template_df, errors="ignore")
template_df = pd.DataFrame.from_dict(config, orient="index").T
template_lib = (template_df, np.expand_dims(selected_schedule, axis=0))
# template_lib = UmiTemplateLibrary.open(
#     Path(
#         "D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Building Energy References/ArchetypalReferences"
#     )
#     / "DefaultTemplateLibrary_1A.json"
# )

# Insert dummy template names
gdf[id_cols["florianpolis"]["template_name_col"]] = "ParametricZoneSettings"
# np.random.choice(
#     [t.Name for t in template_lib.BuildingTemplates], size=gdf.shape[0]
# )

# insert wwrs
gdf[id_cols["florianpolis"]["wwr_col"]] = float(gh_data["ShoeboxConfig"]["wwr"])


Schedule index: 5


In [27]:
from umi.ubem import UBEM

ubem = UBEM(
    gdf=gdf,
    epw=epw,
    **id_cols["florianpolis"],
    template_lib=template_lib,
    sensor_spacing=3,
    shoebox_width=3,
    floor_to_floor_height=3,
    perim_offset=3,
    shoebox_gen_type="edge_unshaded",
)

DEBUG:UMI:INITIAL GEOMETRY CHECK: 0 NaNs, 0 open polygons
DEBUG:UMI:GEOMETRY REPORT
DEBUG:UMI:GEOMETRY, CRS: 0 NaNs, 0 open polygons
DEBUG:UMI:CORES, CRS: 0 NaNs, 41 open polygons
DEBUG:UMI:AREA: 0 NaNs, 0 infs
DEBUG:UMI:CORE_AREAS: 0 NaNs, 0 infs
INFO:Radiation Analysis:Beginning sky matrix extraction...
INFO:Radiation Analysis:Building Count: 93
INFO:Radiation Analysis:Approximate Sensor Count: 6049
INFO:Radiation Analysis:QuadTree Depth: 8
INFO:Radiation Analysis:Initializing azimuths from provided sky...
INFO:Radiation Analysis:Initializing elevations from provided Sky...
INFO:Radiation Analysis:Initializing buildings...
INFO:Radiation Analysis:Extracting edges...
DEBUG:UMI:Built sb df from gis features, 0 NaNs
DEBUG:UMI:Assigned sb dimensions in df from gis features, 0 NaNs
DEBUG:UMI:Added template features, 0 NaNs
DEBUG:UMI:Dropping unnecessary shoeboxes...
DEBUG:UMI:Shoeboxes built... 0 NaNs
INFO:UMI:Processed UMI in 3.72 seconds


In [28]:
# ubem.prepare_epw_features(epw=EPW("C:/ProgramData/Solemma/Common/WeatherData/EPW/USA_MA_Boston-Logan.Intl.AP.725090_TMYx.2004-2018.epw"))
features, schedules, climate = ubem.prepare_for_surrogate()
print("buildings", len(ubem.gis_features_df), "sbs", len(features))

buildings 93 sbs 1022


In [29]:
from ml.surrogate import Surrogate
from ml.predict import predict_ubem
from lightning.pytorch import Trainer
import torch

bucket = "ml-for-bem"
remote_experiment = "full_climate_zone/v7"
local_data_dir = "data/lightning"

registry = "ml-for-building-energy-modeling/model-registry"
model = "Global UBEM Shoebox Surrogate with Combined TS Embedder"
tag = "v3"
resource = "model.ckpt"

surrogate = Surrogate.load_from_registry(registry, model, tag, resource)
surrogate.model.eval()

trainer = Trainer(
    accelerator="auto",
    devices="auto",
    strategy="auto",
    enable_progress_bar=True,
)

c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'target_transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['target_transform'])`.
c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'weather_transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['weather_transform'])`.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, u

In [ ]:
torch.set_float32_matmul_precision("medium")
shoebox_predictions, monthly_predictions, annual_predictions = predict_ubem(
    trainer=trainer,
    surrogate=surrogate,
    features=features,
    schedules=schedules,
    climate=climate,
    batch_size=32,
    apply_cops=False,
)

In [31]:
annual_predictions.describe()

Name,Cooling,Heating
count,93.000000,93.000000
mean,168.837799,49.142276
std,20.106980,35.165712
min,118.318098,3.765823
25%,156.455286,24.136605
50%,169.586641,40.078268
75%,180.082742,61.528392
max,218.218751,151.444340


In [32]:
monthly_predictions.head()

Name           Cooling                                                         \
            2018-01-01 2018-02-01 2018-03-01 2018-04-01 2018-05-01 2018-06-01   
building_id                                                                     
0             4.535458   6.002228   9.153663  14.420216  19.858925  26.447049   
1             1.805474   2.722429   4.550512   8.888773  14.714762  23.997343   
2             0.631394   1.424839   2.879859   7.587852  14.484407  27.785799   
3             2.560088   3.705572   6.084950  11.385499  17.682210  26.371654   
4             0.227550   0.677045   1.232711   3.970586   9.733421  22.426092   

Name                                                     ...    Heating  \
            2018-07-01 2018-08-01 2018-09-01 2018-10-01  ... 2018-03-01   
building_id                                              ...              
0            31.451484  30.309914  22.721187  15.244874  ...   0.572919   
1            31.377067  30.032457  20.284910  10.774946  ...   4.340377   
2            37.937318  36.070107  21.888065   9.628818  ...   7.776507   
3            33.091536  31.760703  21.945344  12.603615  ...   2.207517   
4            32.173011  30.227935  17.158100   6.234859  ...  11.675836   

Name                                                                           \
            2018-04-01 2018-05-01 2018-06-01 2018-07-01 2018-08-01 2018-09-01   
building_id                                                                     
0             0.038899  -0.024420  -0.020415  -0.020082  -0.018092  -0.019666   
1             0.466511  -0.028693  -0.024842  -0.022203  -0.019966  -0.021144   
2             0.966353  -0.025439  -0.025940  -0.024834  -0.021619  -0.020184   
3             0.212426  -0.027474  -0.023514  -0.021955  -0.019566  -0.020335   
4             1.426944  -0.031173  -0.030126  -0.024434  -0.022083  -0.022849   

Name                                          
            2018-10-01 2018-11-01 2018-12-01  
building_id                                   
0            -0.003006   0.295710   1.100824  
1             0.106169   2.178456   6.281199  
2             0.313027   4.220110  11.262413  
3             0.035563   1.139890   3.561051  
4             0.283401   6.087343  15.815843  

[5 rows x 24 columns]

## Compare

In [39]:
tfa = ubem.gis_features_df.bldg_footprint_area * ubem.gis_features_df.floor_count

In [49]:
# px.bar(x=annual_predictions.index, y=annual_predictions["Cooling"], title="Surrogate")
fig = px.bar(
    x=annual_predictions.index,
    y=[
        annual_predictions.Cooling.divide(COP_COOLING),
        # umi_df[umi_df["load"] == "cooling"]["annual"].divide(tfa, axis=0),
        CS_building_results['cooling'].sum(axis=1),
    ],
    title="Florianopolis Cooling with COP: CS (red), surrogate (blue)",
    barmode="group"
)
fig.show()

In [44]:

fig = px.bar(
    x=annual_predictions.index,
    y=[
        annual_predictions.Heating.divide(COP_HEATING),
        # umi_df[umi_df["load"] == "heating"]["annual"].divide(tfa, axis=0),
        CS_building_results['heating'].sum(axis=1),
    ],
    title="Florianopolis Heating with COP: CS (red), surrogate (blue)",
    barmode="group"
)
fig.show()

In [38]:
umi_df_mon_norm = get_monthly(umi_df).groupby("load").sum().divide(tfa.sum())
umi_df_mon_norm

(186, 8760)


,january,february,march,april,may,june,july,august,september,october,november,december
load,,,,,,,,,,,,
cooling,0.000000,0.00000,0.000000,0.000000,0.0,0.720345,2.467,3.391396,2.156917,0.969094,0.645356,0.395199
heating,1.176728,5.15668,3.361244,0.533238,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000


In [66]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = list(month_map.keys()),
        y = monthly_predictions["Cooling"].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()*COP_COOLING),
        mode='lines+markers',
        name = "Surrogate, Cooling"
    )
)
fig.add_trace(
    go.Scatter(
        x = list(month_map.keys()),
        y = monthly_predictions["Heating"].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()*COP_HEATING),
        mode='lines+markers',
        name = "Surrogate, Heating"
    )
)
fig.add_trace(
    go.Scatter(
        x = list(month_map.keys()),
        y = CS_results["cooling"],
        mode='lines+markers',
        name = "CS context, Cooling"
    )
)
fig.add_trace(
    go.Scatter(
        x = list(month_map.keys()),
        y = CS_results["heating"],
        mode='lines+markers',
        name = "CS context, Heating"
    )
)
fig.add_trace(
    go.Scatter(
        x = list(month_map.keys()),
        y = CS_building_results['cooling'].multiply(list(tfa), axis=0).sum(axis=0).divide(tfa.sum()),
        mode='lines+markers',
        name = "CS no-shad, Cooling"
    )
)
fig.add_trace(
    go.Scatter(
        x = list(month_map.keys()),
        y = CS_building_results['heating'].multiply(list(tfa), axis=0).sum(axis=0).divide(tfa.sum()),
        mode='lines+markers',
        name = "CS no-shad, Heating"
    )
)

# colors = []
# for i in range(4):
#     fig.data[i].line.color = colors[i]
fig.update_layout(
    title="Surrogate, CS context, and CS-no shading average normalized heating and cooling",
    autosize=False,
    width=800,
    height=600,
)
fig.show()

# Shallow Upgrade

## CS Results

In [18]:
# CS with shading
with open(Path(shallow_results_dir, "results-out-CS_boston_shallow.json"), "r") as f:
    CS_shallow_results = json.load(f)
CS_shallow_results = pd.DataFrame.from_dict(CS_results)
CS_shallow_results

,heating,cooling
0,22.477821,0.224322
1,16.568589,0.334789
2,12.775908,0.479481
3,4.114498,1.184509
4,1.032315,2.731751
5,0.269734,4.966134
6,0.015241,6.698250
7,0.024786,6.592109
8,0.280518,3.864128
9,2.737289,1.620085


In [19]:
# CS individual buildings
CS_shallow_building_results = {}
for i in range(93):
    with open(Path(shallow_results_dir) / "SingleBuildings" / f"results-out-CS_boston_shallow{i}.json", "r") as f:
        r = json.load(f)
        res_dict = {}
        for l in ["heating", "cooling"]:
            res_dict.update({(l, k): r for k, r in zip(month_map.keys(), r[l])})
        CS_shallow_building_results[i] = res_dict
CS_shallow_building_results = pd.DataFrame.from_dict(CS_shallow_building_results).T
CS_shallow_building_results.head()

heating                                                                \
     january   february      march     april       may      june      july   
0  10.942497   7.987217   6.431928  2.759895  1.175352  0.312455  0.040475   
1  12.362149   8.167249   5.917305  1.485072  0.227303  0.039505  0.000232   
2  10.594664   6.727063   4.563097  0.941800  0.109507  0.018666  0.000089   
3  36.736075  26.438094  21.102043  7.277118  2.145279  0.423855  0.018595   
4   1.546287   0.789188   0.245070  0.029862  0.000653  0.000049  0.000000   

                                 ...   cooling                                 \
     august september   october  ...     march     april       may       june   
0  0.063708  0.531463  2.409826  ...  2.141935  3.154461  4.697142   6.304154   
1  0.000000  0.068340  1.253316  ...  3.396675  5.435102  7.977695  10.812200   
2  0.000000  0.034984  0.733368  ...  3.474549  5.748709  8.351923  11.267772   
3  0.007935  0.541173  5.683254  ...  2.735369  4.560194  7.550199  11.534453   
4  0.000000  0.000025  0.012384  ...  1.209989  2.627393  3.930558   5.324631   

                                                                 
        july     august september   october  november  december  
0   7.450702   7.503211  4.870723  3.229816  2.422535  1.144982  
1  12.905640  12.884518  8.287581  5.326634  3.605414  1.489734  
2  13.476894  13.448886  8.753028  5.678030  3.758960  1.488737  
3  14.523765  14.336669  8.674175  4.822938  3.291734  1.352012  
4   6.409222   6.386708  4.393837  2.838789  1.587613  0.560443  

[5 rows x 24 columns]

In [20]:
CS_shallow_building_results.loc[np.isnan(CS_shallow_building_results['heating']['january'])]

Empty DataFrame
Columns: [(heating, january), (heating, february), (heating, march), (heating, april), (heating, may), (heating, june), (heating, july), (heating, august), (heating, september), (heating, october), (heating, november), (heating, december), (cooling, january), (cooling, february), (cooling, march), (cooling, april), (cooling, may), (cooling, june), (cooling, july), (cooling, august), (cooling, september), (cooling, october), (cooling, november), (cooling, december)]
Index: []

[0 rows x 24 columns]

## Model Results

In [21]:
COP_COOLING = 3.32337
COP_HEATING = 0.75

In [48]:
shallow_features.VentilationPerPerson

0       0.0
1       0.0
2       0.0
4       0.0
5       0.0
       ... 
1885    0.0
1886    0.0
1891    0.0
1907    0.0
1915    0.0
Name: VentilationPerPerson, Length: 1022, dtype: object

In [22]:
shallow_features, schedules, climate = ubem.prepare_for_surrogate()
print("buildings", len(ubem.gis_features_df), "sbs", len(features))

buildings 93 sbs 1022


In [23]:
# Apply upgrades
shallow_features["WindowUValue"] = 1.0
shallow_features["WindowSHGC"] =0.65
shallow_features["HeatingSetpoint"] = 18 # why are these different???
shallow_features["RecoverySettings"] = 1

In [26]:
torch.set_float32_matmul_precision("medium")
shallow_shoebox_predictions, shallow_monthly_predictions, shallow_annual_predictions = predict_ubem(
    trainer=trainer,
    surrogate=surrogate,
    features=shallow_features,
    schedules=schedules,
    climate=climate,
    batch_size=32,
    apply_cops=False,
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   0%|          | 0/32 [00:00<?, ?it/s]

c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Convolution.cpp:1009.)
  return F.conv1d(input, weight, bias, self.stride,


Predicting DataLoader 0: 100%|██████████| 32/32 [01:59<00:00,  0.27it/s]


In [46]:
import plotly.graph_objects as go

fig = go.Figure()


# fig.add_trace(
#     go.Scatter(
#         x = umi_df_mon_norm.columns,
#         y = shallow_monthly_predictions["Cooling"].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()*COP_COOLING),
#         mode='lines+markers',
#         name = "Surrogate, Cooling",
#         line=dict(color="blue")
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x = umi_df_mon_norm.columns,
#         y = shallow_monthly_predictions["Heating"].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()*COP_HEATING),
#         mode='lines+markers',
#         name = "Surrogate, Heating",
#         line=dict(color="red")
#     )
# )
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_shallow_results["cooling"],
        mode='lines+markers',
        name = "CS context, Cooling",
        line=dict(color="blue")
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_shallow_results["heating"],
        mode='lines+markers',
        name = "CS context, Heating",
        line=dict(color="red")
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_shallow_building_results['cooling'].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()),
        mode='lines+markers',
        name = "CS no-shad, Cooling",
        line=dict(color="blue", dash='dash')
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_shallow_building_results['heating'].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()),
        mode='lines+markers',
        name = "CS no-shad, Heating",
        line=dict(color="red", dash='dash')
    )
)

# colors = []
# for i in range(4):
#     fig.data[i].line.color = colors[i]
fig.update_layout(
    title="CS context CS-no shading normalized heating and cooling",
    autosize=False,
    width=800,
    height=600,
)
fig.show()

In [45]:
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = shallow_monthly_predictions["Cooling"].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()*COP_COOLING),
        mode='lines+markers',
        name = "Surrogate, Cooling",
        line=dict(color="blue")
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = shallow_monthly_predictions["Heating"].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()*COP_HEATING),
        mode='lines+markers',
        name = "Surrogate, Heating",
        line=dict(color="red")
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_shallow_building_results['cooling'].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()),
        mode='lines+markers',
        name = "CS no-shad, Cooling",
        line=dict(color="blue", dash='dash')
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_shallow_building_results['heating'].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()),
        mode='lines+markers',
        name = "CS no-shad, Heating",
        line=dict(color="red", dash='dash')
    )
)

# colors = []
# for i in range(4):
#     fig.data[i].line.color = colors[i]
fig.update_layout(
    title="Surrogate and CS-no shading normalized heating and cooling",
    autosize=False,
    width=800,
    height=600,
)
fig.show()

In [183]:
# Annual comparison
x = [["Baseline", "Upgrade"]*3, ['Surrogate', "CS no-shad", 'CS Context']*2]
fig = go.Figure(data=[
    go.Bar(
        name='Cooling', 
        x=x, 
        y=[
            annual_predictions["Cooling"].sum()/COP_COOLING/93,
            CS_building_results['cooling'].sum().sum()/93,
            CS_results['cooling'].sum(),
            shallow_annual_predictions["Cooling"].sum()/COP_COOLING/93,
            CS_shallow_building_results['cooling'].sum().sum()/93,
            CS_shallow_results['cooling'].sum(),
            ]),
    go.Bar(
        name='Heating', 
        x=x, 
        y=[
            annual_predictions["Heating"].sum()/COP_HEATING/93,
            CS_building_results['heating'].sum().sum()/93,
            CS_results['heating'].sum(),
            shallow_annual_predictions["Heating"].sum()/COP_HEATING/93,
            CS_shallow_building_results['heating'].sum().sum()/93,
            CS_shallow_results['heating'].sum(),
            ]
        ),
])
fig.update_layout(
    barmode='stack',
    autosize=False,
    width=800,
    height=600,
    # update_x
    )